In [1]:
import random
import heapq
import numpy as np

# Simulation parameters
SIMULATION_TIME = 1000  # minutes
NUM_DOCTORS = 3
MEAN_INTERARRIVAL_TIME = 5  # minutes
SERVICE_TIMES = {1: 15, 2: 10, 3: 5}  # average service times by triage level
TRIAGE_PROBS = [0.2, 0.5, 0.3]  # high, medium, low priority

# Event queue: (time, event_type, patient)
event_queue = []

# Priority queue for waiting patients: (priority, arrival_time, patient_id)
waiting_patients = []

# Doctors: list of (available_time, doctor_id)
doctors = [(0, i) for i in range(NUM_DOCTORS)]

# Tracking
patient_id_counter = 0
all_wait_times = []
doctor_busy_times = [0] * NUM_DOCTORS


def generate_interarrival_time():
    return np.random.exponential(MEAN_INTERARRIVAL_TIME)


def assign_triage_level():
    return np.random.choice([1, 2, 3], p=TRIAGE_PROBS)


def generate_service_time(triage_level):
    return np.random.exponential(SERVICE_TIMES[triage_level])


def schedule_event(time, event_type, patient):
    heapq.heappush(event_queue, (time, event_type, patient))


def find_available_doctor(current_time):
    for i in range(len(doctors)):
        if doctors[i][0] <= current_time:
            return i
    return None


# Start simulation by scheduling the first arrival
current_time = 0
first_patient = {
    'id': patient_id_counter,
    'arrival_time': 0,
    'triage_level': assign_triage_level()
}
schedule_event(0, 'arrival', first_patient)

while current_time < SIMULATION_TIME and event_queue:
    current_time, event_type, patient = heapq.heappop(event_queue)

    if event_type == 'arrival':
        print(f"Time {current_time:.2f}: Patient {patient['id']} arrives with triage level {patient['triage_level']}")
        patient_id_counter += 1

        # Add patient to the waiting queue (min-heap by priority and arrival)
        heapq.heappush(waiting_patients, (patient['triage_level'], patient['arrival_time'], patient['id'], patient))

        # Schedule next arrival
        next_arrival_time = current_time + generate_interarrival_time()
        if next_arrival_time < SIMULATION_TIME:
            new_patient = {
                'id': patient_id_counter,
                'arrival_time': next_arrival_time,
                'triage_level': assign_triage_level()
            }
            schedule_event(next_arrival_time, 'arrival', new_patient)

        # Attempt to serve waiting patient
        doc_idx = find_available_doctor(current_time)
        if doc_idx is not None and waiting_patients:
            _, arrival_time, _, next_patient = heapq.heappop(waiting_patients)
            wait_time = current_time - arrival_time
            all_wait_times.append(wait_time)
            service_time = generate_service_time(next_patient['triage_level'])
            finish_time = current_time + service_time
            doctors[doc_idx] = (finish_time, doc_idx)
            doctor_busy_times[doc_idx] += service_time
            print(f"  Doctor {doc_idx} starts serving Patient {next_patient['id']} (waited {wait_time:.2f} mins)")
            schedule_event(finish_time, 'departure', next_patient)

    elif event_type == 'departure':
        print(f"Time {current_time:.2f}: Patient {patient['id']} departs")

        # Free up doctor and check for next patient
        doc_idx = find_available_doctor(current_time)
        if doc_idx is not None and waiting_patients:
            _, arrival_time, _, next_patient = heapq.heappop(waiting_patients)
            wait_time = current_time - arrival_time
            all_wait_times.append(wait_time)
            service_time = generate_service_time(next_patient['triage_level'])
            finish_time = current_time + service_time
            doctors[doc_idx] = (finish_time, doc_idx)
            doctor_busy_times[doc_idx] += service_time
            print(f"  Doctor {doc_idx} starts serving Patient {next_patient['id']} (waited {wait_time:.2f} mins)")
            schedule_event(finish_time, 'departure', next_patient)

# Performance metrics
print("\nSimulation Complete.\n")
print(f"Total patients served: {len(all_wait_times)}")
print(f"Average wait time: {np.mean(all_wait_times):.2f} minutes")
for i, busy_time in enumerate(doctor_busy_times):
    utilization = (busy_time / SIMULATION_TIME) * 100
    print(f"Doctor {i} utilization: {utilization:.2f}%")


Time 0.00: Patient 0 arrives with triage level 2
  Doctor 0 starts serving Patient 0 (waited 0.00 mins)
Time 3.82: Patient 1 arrives with triage level 3
  Doctor 1 starts serving Patient 1 (waited 0.00 mins)
Time 7.20: Patient 2 arrives with triage level 1
  Doctor 2 starts serving Patient 2 (waited 0.00 mins)
Time 10.25: Patient 0 departs
Time 10.54: Patient 1 departs
Time 11.20: Patient 3 arrives with triage level 2
  Doctor 0 starts serving Patient 3 (waited 0.00 mins)
Time 11.87: Patient 2 departs
Time 12.46: Patient 4 arrives with triage level 1
  Doctor 1 starts serving Patient 4 (waited 0.00 mins)
Time 18.34: Patient 3 departs
Time 23.44: Patient 5 arrives with triage level 2
  Doctor 0 starts serving Patient 5 (waited 0.00 mins)
Time 25.13: Patient 5 departs
Time 27.50: Patient 6 arrives with triage level 2
  Doctor 0 starts serving Patient 6 (waited 0.00 mins)
Time 30.86: Patient 7 arrives with triage level 3
  Doctor 2 starts serving Patient 7 (waited 0.00 mins)
Time 33.19: P